In [1]:
import os

from dataclasses import dataclass

import numpy as np
import pandas as pd

import andes
import ams

import cosim

In [2]:
andes.config_logger(stream_level=50)
ams.config_logger(stream_level=50)

In [3]:
case_path = "./../cases"
res_path = "./../results"
addfile = case_path + '/IL200_dyn_db2.xlsx'

# --- file loading ---
curve = pd.read_csv(case_path + '/CurveInterp.csv')

In [4]:
for hour in [0]:
    hour = 0
    curve_hour = curve.iloc[hour * 3600:(hour+1) * 3600].reset_index(drop=True)

    res_csv = os.path.join(res_path, f'case1_H{hour}.csv')

    sp = ams.load(case_path + '/IL200_rted.xlsx',
                  setup=True, no_output=True,
                  default_config=True)
    sa = sp.to_andes(addfile=addfile,
                     setup=True, no_output=True,
                     default_config=True)

    cosim = cosim.CoSimulation(sp=sp, sa=sa,
                               curve=curve_hour,
                               addfile=addfile,
                               res_csv=res_csv,)

    cosim.loop()

Generating code for 1 models on 12 processes.
--- RTED: 0 ---
rows: 0 - 300
Generating code for 1 models on 12 processes.


/Users/jinningwang/work/demo/demo/deadband/notes/cosim.py:111: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.link = self.sp.dyn.link.copy().fillna(False)
/Users/jinningwang/work/andes/andes/routines/tds.py:372: RuntimeWarning: invalid value encountered in scalar divide
  perc = round((dae.t - config.t0) / (config.tf - config.t0) * 100, 2)


ANDES Load: 14.76 MW,
Load: 1222.84 MW, Gen.: 1222.84 MW Cost: 68925.86 $
Watchdog: Second: 0
Watchdog: Second: 100
Watchdog: Second: 200
--- RTED: 1 ---
rows: 300 - 600
Generating code for 1 models on 12 processes.


/Users/jinningwang/work/demo/demo/deadband/notes/cosim.py:111: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  self.link = self.sp.dyn.link.copy().fillna(False)
/Users/jinningwang/work/andes/andes/routines/tds.py:372: RuntimeWarning: invalid value encountered in scalar divide
  perc = round((dae.t - config.t0) / (config.tf - config.t0) * 100, 2)


ANDES Load: 14.76 MW,
Load: 1216.85 MW, Gen.: 1216.85 MW Cost: 68465.07 $
Watchdog: Second: 300


RuntimeError: ANDES exited with 1 at 303s